In [145]:
import numpy as np
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D
from uszipcode import ZipcodeSearchEngine
from geopy.geocoders import Nominatim

import pandas as pd
import seaborn.apionly as sns
from datetime import date, datetime
from haversine import haversine

# statistics package
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats
from random import randint
# packages for mapping
from mpl_toolkits.basemap import Basemap

# packages for interactive graphs
from ipywidgets import widgets, interact
from IPython.display import display
from copy import deepcopy as copy
import time
from geopy.geocoders import Nominatim

from time import sleep
import requests
import time,datetime
import os
%matplotlib inline

In [101]:
findzip = ZipcodeSearchEngine()

In [263]:
historical_data = pd.read_csv('clean_train_data.csv')

In [264]:
historical_data.head()

,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,...,pickup_date,dropoff_date,distance,speed,day_interval,waiting_time,waiting_penalty,pickup_zipcode,dropoff_zipcode,weather
0,BE530E79CB7E459DEF5BBDF2F319EDC1,0002555BBE359440D6CEB34B699D3932,CMT,1,N,2013-01-01 19:00:00,2013-01-01 19:16:33,1,992,7.9,...,2013-01-01,2013-01-01,11.499047,41.730414,2,0.000000,0.000000,11371,10016,Rain
1,BE530E79CB7E459DEF5BBDF2F319EDC1,0002555BBE359440D6CEB34B699D3932,CMT,1,N,2013-01-01 19:52:35,2013-01-01 19:58:07,2,331,1.5,...,2013-01-01,2013-01-01,3.178577,34.570622,2,2016.523623,2.800727,10174,10009,Rain
2,BE530E79CB7E459DEF5BBDF2F319EDC1,0002555BBE359440D6CEB34B699D3932,CMT,1,N,2013-01-01 21:23:24,2013-01-01 21:40:06,1,1001,9.2,...,2013-01-01,2013-01-01,13.149924,47.292434,2,6736.024510,9.355590,11371,10009,Rain
3,BE530E79CB7E459DEF5BBDF2F319EDC1,0002555BBE359440D6CEB34B699D3932,CMT,1,N,2013-01-01 23:01:07,2013-01-01 23:05:41,2,273,1.7,...,2013-01-01,2013-01-01,0.475662,6.272460,2,6067.954056,8.427714,11369,11369,Rain
4,BE530E79CB7E459DEF5BBDF2F319EDC1,0002555BBE359440D6CEB34B699D3932,CMT,1,N,2013-01-01 23:40:33,2013-01-01 23:53:54,1,801,7.5,...,2013-01-01,2013-01-01,6.674613,29.998262,2,14321.085900,19.890397,11371,10128,Rain


In [265]:
historical_data.keys()

Index(['medallion', 'hack_license', 'vendor_id', 'rate_code',
       'store_and_fwd_flag', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'trip_time_in_secs', 'trip_distance',
       'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'fare_amount', 'payment_type', 'surcharge',
       'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount', 'pickup_day',
       'pickup_weekday', 'pickup_hour', 'pickup_minute', 'pickup_time',
       'dropoff_hour', 'pickup_date', 'dropoff_date', 'distance', 'speed',
       'day_interval', 'waiting_time', 'waiting_penalty', 'pickup_zipcode',
       'dropoff_zipcode', 'weather'],
      dtype='object')

## Best Revenue from Historical Data

In [266]:
best_revenue = historical_data.groupby(['hack_license','pickup_date'])['total_amount'].sum()

In [267]:
np.max(best_revenue.reset_index()['total_amount'])

895.7299999999999

## Preprocess

In [268]:
historical_data["dropoff_minute"] = pd.to_datetime(historical_data["dropoff_datetime"]).apply(lambda x: x.minute)
historical_data["dropoff_time"] =historical_data["dropoff_hour"] + (historical_data["dropoff_minute"] / 60)

## Reinforcement Model

In [269]:
class TaxiWorld():
    def __init__(self):
        self.waiting_penalty = 5.00
        pass
                
    def getReward(self, row):
        if row['waiting_time']/3600<=2:
            return row['total_amount'] - row['waiting_time']/3600*self.waiting_penalty
        else:
            return row['total_amount']
        
    
    def set_waiting_penalty(self, penalty):
        self.waiting_penalty = penalty
        
    def getState(self):
        print('Current State')
        return self.s
    
    def setState(self, s):
        self.s = s
        
    def getStateSize(self, trip_and_fare):
        return len(trip_and_fare['zip_code'].unique())
    
    def getZipCodes(self, trip_and_fare):
        return self.zip_codes
    
    def getActionSize(self):
        return len(self.actions)

    def nextAction(self, s):
        print('Calculate Next Action based on state')
        
    def check_Q_table(self, s):
        if s not in self.Q:
            self.Q[s] = dict((action, 0.0) for action in self.actions)
            

In [270]:
env = TaxiWorld()

In [271]:
class TaxiRevenue:
    def __init__(self, env, penalty=5.0, Q_table_file_name='saved_Q.json'):
        self.env = env
        self.env.set_waiting_penalty(penalty)
        self.Q = dict()
        self.dropPickMap= dict()
        self.Q_table_file_name = Q_table_file_name
        self.load_Q()
        
    def load_Q(self):
        if os.path.isfile(self.Q_table_file_name):
            with open(self.Q_table_file_name, 'r') as fp:
                self.Q = json.load(fp)
    
    def store_Q(self):
        with open(self.Q_table_file_name, 'w') as fp:
            json.dump(self.Q, fp)
        
    def generate_key(self, row):
        return str(row['pickup_zipcode']) + '_'  + str(row['pickup_weekday']) + '_' + str(row['day_interval']) + '_' + str(row['weather'])
    
    def get_tuples(self, key):
        tuples = key.split('_')
        return {'pickup_zipcode': tuples[0],
                   'pickup_weekday': tuples[1],
                   'day_interval': tuples[2],
                   'weather': tuples[3]}
    
    def get_meantriptime(self, data):
        map=historical_data.groupby(['pickup_zipcode', 'dropoff_zipcode'])['trip_time_in_secs'].mean()
        return map.reset_index()
    
    def isValidPickup(self, taxidata, dropPickMap):
        key=self.generate_key(taxidata)
        if key in dropPickMap:
            return True
        else:
            return False
        
    def generate_dropPickMap(self, data):
        for index,row in data.T.iteritems():
            key = self.generate_key(row)
            if key not in self.dropPickMap:
                self.dropPickMap[key] = []
            if str(row['dropoff_zipcode']) not in self.dropPickMap[key]:
                self.dropPickMap[key].append(str(row['dropoff_zipcode']))
        return self.dropPickMap
    
    def getDropoff(self, key, dropPickMap):
        posDropoff= dropPickMap[key]
        print("dropoffs",posDropoff )
        rand=randint(0, len(posDropoff)-1)
        return posDropoff[rand]
         
    def getDayInterval(self,hour):
        if hour>=6 and hour<12:
            return 0
        elif hour>=12 and hour<18:
            return 1
        elif hour>=18 and hour<24:
            return 2
        else:
            return 3
        
    def getAction(self, taxidata, _Q,dropPickMap, const):
        best_reward=0
        #best_key = 0
        s = self.generate_key(taxidata)
        best_key=s
        #print(taxidata)
        if s in _Q:
            best_reward=_Q[s][0]*_Q[s][1]/const
        
        zipcode=findzip.by_zipcode(taxidata['dropoff_zipcode'])
        temp=taxidata
        possible_pickup=[]
        returns=100
        while not possible_pickup:
            pickup_res = findzip.by_coordinate(zipcode['Latitude'], zipcode['Longitude'], radius=10, returns=returns)
            for i in range(len(pickup_res)):
                temp['pickup_zipcode']=pickup_res[i]['Zipcode']
                #print(temp['pickup_zipcode'])
                if self.isValidPickup(temp, dropPickMap):
                    possible_pickup.append(pickup_res[i]['Zipcode'])

            print("valid pickup", possible_pickup)
            returns+=10
        
        for key in _Q:
            tuples=self.get_tuples(key)
               
            
            if tuples['pickup_zipcode'] in possible_pickup and tuples['weather']==str(taxidata['weather']) and tuples['pickup_weekday']==str(taxidata['pickup_weekday']) and tuples['day_interval']==str(taxidata['day_interval']):
            
                tempReward=_Q[key][0]*_Q[key][1]/const
                
                if tempReward>best_reward:
                    #print("in temp reward: ",tempReward)
                    #print("in best reward: ",best_reward)
                    best_key=key
                    best_reward= tempReward
                    #print(self.get_tuples(best_key))
                    
        return best_key
    
    def build_Q_table(self, data):
        for index,row in data.T.iteritems():
            key = self.generate_key(row)
            reward = self.env.getReward(row)
            if key not in self.Q:
                self.Q[key] = [reward,1]
            else:
                self.Q[key] = [self.Q[key][0] +reward, self.Q[key][1] + 1]
                
    def greedy(self, s):
        return np.argmax(self.Q[s[0]]) 

    def epsilon_greed(self, epsilon, s):
        if np.random.rand() < epsilon:
            return np.random.randint(self.n_a)
        else:
            return self.greedy(s)
        
    def train(self, trip_and_fare, Q_table_file_name =  'saved_Q.json'):
        self.Q_table_file_name = Q_table_file_name
        self.build_Q_table(trip_and_fare)
        self.store_Q()
        return self.Q
        
    def distance(self, lat1, lon1, lat2, lon2):
        """calculates the Manhattan distance between 2 points
            using their coordinates

        Parameters
        ----------
        lat1: float
            latitude of first point

        lon1: float
            longitude of first point

        lat2: float
            latitude of second point

        lon2: float
            longitude of second point

        Returns
        -------
        d: float
            The Manhattan distance between the two points in kilometers

        """

        d = haversine((lat1, lon1), (lat2, lon1)) + haversine((lat2, lon1), (lat2, lon2))
        return d
    
    def find_distance(self, zip1, zip2):
        zipcode1=findzip.by_zipcode(zip1)
        zipcode2=findzip.by_zipcode(zip2)
        dis=self.distance(zipcode1['Latitude'], zipcode1['Longitude'],zipcode2['Latitude'], zipcode2['Longitude'])
        return dis
    
    
    def test(self, taxidata, _Q, dropPickMap,meanTrip, maxstep=1000):
        # TODO: Add your test routine that exmines learned policy
        #      Using the learned policy, generate the trajectory of 
        curr_dropoff_hour=taxidata['dropoff_time']
        #curr_pickup=taxidata['dropoff_time']
        end_time=curr_dropoff_hour+ 12
        
        trace = []
        rewards = []
        
        while curr_dropoff_hour <= end_time:
            
            best_key=self.getAction( taxidata, _Q,dropPickMap, 1000)
            pickup_tuple= self.get_tuples(best_key)
            
            if int(pickup_tuple['pickup_zipcode'])!=taxidata['dropoff_zipcode']:
                dis=self.find_distance(taxidata['dropoff_zipcode'], int(pickup_tuple['pickup_zipcode']))
                curr_dropoff_hour+= dis/taxidata['speed']
                print("roaming time", dis/taxidata['speed'] )
                
            dropoff=self.getDropoff(best_key, dropPickMap)
                
            print("going from ",pickup_tuple['pickup_zipcode'], " to ",  dropoff, " at ", curr_dropoff_hour)
            #curr_dropoff=curr_dropoff+meanTrip[meanTrip['pickup_zipcode']]
            #print(pickup_tuple['pickup_zipcode'])
            temptrip = meanTrip[(meanTrip['pickup_zipcode']==int(pickup_tuple['pickup_zipcode'])) & (meanTrip['dropoff_zipcode']==int(dropoff))]
            meanTripTime= temptrip.iloc[0]['trip_time_in_secs']
            #print(meanTripTime)
            
            curr_dropoff_hour=curr_dropoff_hour+meanTripTime/3600
            taxidata['dropoff_zipcode']=int(dropoff)
            taxidata['day_interval']=self.getDayInterval(curr_dropoff_hour)

## Train the Model

In [272]:
taxiRevenue = TaxiRevenue(env, 5.0, 'saved_Q.json')

In [273]:
_Q = taxiRevenue.train(historical_data)

In [274]:
len(_Q)

5496

## Generate Test Data

In [275]:
dropPickMap=taxiRevenue.generate_dropPickMap(historical_data)
meanTrip= taxiRevenue.get_meantriptime(historical_data)

## Test the Model

In [276]:
taxiRevenue.test(historical_data.iloc[14000], _Q, dropPickMap, meanTrip)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


valid pickup ['10014', '10011', '10012', '10013', '10003', '10199', '10119', '10001', '10282', '10010', '10278', '10018', '10007', '10009', '10016', '10036', '10006', '10165', '10168', '10173', '10280', '10002', '10170', '10038', '10103', '10174', '10020', '10167', '10112', '10004', '10111', '10017', '10019', '10154', '10005', '10022', '10069', '11109', '11201', '10065', '11222', '10023', '10044', '11211', '10021', '11101', '10075', '10024', '11205', '10028', '11231', '11217', '10128', '11106', '11238', '11104', '10025', '11102', '11215', '11216', '10029', '11378', '11103', '10026', '11377', '10115', '10035', '10027']
roaming time 0.2404467782275536
dropoffs ['10003', '10119', '10016', '10165', '10013', '10018', '10010', '10014', '10111', '11369', '10017', '10011', '10451', '10115', '10012', '10019', '10173', '10022', '10020', '10006', '10036', '7114', '10167', '10004', '11436', '10075', '10023', '10174', '10199', '10065', '10128', '10282', '10001', '11430', '10032', '10028', '11238', 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:195: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


valid pickup ['10010', '10003', '10016', '10009', '10168', '10165', '10174', '10170', '10119', '10173', '10017', '10167', '10199', '10012', '10018', '11109', '10001', '10154', '10112', '10111', '10014', '10020', '10022', '10011', '10036', '10002', '11222', '10013', '10044', '10278', '10065', '10103', '11101', '10019', '10007', '10282', '11211', '10038', '10021', '10069', '10006', '10023', '10075', '11201', '10280', '10004', '10005', '10028', '11106', '11104', '10024', '11205', '10128', '11102', '11378', '11217', '11103', '11377', '10029', '11238', '10025', '11216', '11231', '10026', '10035', '11105', '11215', '11370', '10115', '11233', '10027', '11373', '11372']
roaming time 0.18642553001172626
dropoffs ['10003', '10119', '10016', '10165', '10013', '10018', '10010', '10014', '10111', '11369', '10017', '10011', '10451', '10115', '10012', '10019', '10173', '10022', '10020', '10006', '10036', '7114', '10167', '10004', '11436', '10075', '10023', '10174', '10199', '10065', '10128', '10282',

valid pickup ['11374', '11373', '11378', '11385', '11375', '11377', '11237', '11372', '11104', '11415', '11368', '11370', '11369', '11206', '11233', '11103', '11419', '11101', '11222', '11106', '11435', '11371', '11211', '11205', '11105', '11109', '11420', '11102', '11216', '10044', '11238', '10017', '10009', '10065', '11436', '10021', '11225', '10174', '10075', '10022', '10010', '10028', '10002', '10170', '10154', '10167', '10168', '10016', '10165', '10003', '11201', '11217', '10173', '10128', '10103', '10035', '10111', '10112', '10454', '10020', '10029', '10012']
roaming time 1.1083329386009881
dropoffs ['10021', '10023', '10024', '10006', '10017', '10014', '7030', '10003', '10016', '10036', '10010', '10020', '10025', '10028', '11377', '10012', '10280', '10075', '10002', '10019', '10065', '10119', '11201', '11217', '11232', '10013', '11430', '10128', '10011', '10111', '10069', '10278', '10004', '10018', '10030', '10033', '10007', '10037', '10009', '10199', '10022', '10035', '10282', 

valid pickup ['10024', '10023', '10025', '10069', '10128', '10028', '10075', '10021', '10019', '10029', '10065', '10026', '10115', '10020', '10022', '10111', '10112', '10154', '10027', '10036', '10167', '10173', '10044', '10170', '10018', '10165', '10017', '10174', '10168', '10035', '11102', '10001', '10199', '10119', '10016', '10030', '10037', '11109', '11106', '10031', '10010', '10011', '11101', '10039', '11105', '11103', '10003', '10014', '11104', '10009', '10032', '11222', '10012', '11370', '10013', '11377', '10002', '10278', '10459', '10282', '11371', '10007', '10103', '11211']
roaming time 0.7286728934243034
dropoffs ['10012', '11221', '11201', '10119', '10018', '10028', '10009', '7114', '10023', '10022', '10003', '10010', '11238', '11206', '10017', '10075', '10128', '10020', '10014', '10006', '10280', '10016', '10040', '11430', '10002', '10011', '10199', '10019', '10024', '10065', '11211', '10036', '10004', '11214', '11205', '10278', '11104', '11222', '11369', '11231', '10033', 

## Merge Train Data

In [277]:
train_data1 = pd.read_csv('clean_data.csv')
train_data2 = pd.read_csv('clean_data_2.csv')
train_data3 = pd.read_csv('clean_data_3.csv')
train_data4 = pd.read_csv('clean_data_4.csv')
train_data5 = pd.read_csv('clean_data_5.csv')
frames = [train_data1, train_data2, train_data3, train_data4, train_data5]
result = pd.concat(frames)
result.to_csv('clean_train_data.csv', index=False)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [278]:
result.shape

(2905906, 38)